In [2]:
# add libraries
import requests
import json
import pandas as pd
import os

# Data Acquisition
### Grab data from different API sources (Spotify, Genius, Billboard 100)

In [7]:
# function for grabbing api key
def get_file_contents(filename):
    """ Given a filename,
        return the contents of that file
    """
    try:
        with open(filename, 'r') as f:
            # It's assumed our file contains a single line,
            # with our API key
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)

#### Spotify

In [8]:
spotify_filename = "../spotify_api"

spotify_key = get_file_contents(spotify_filename)

spotify_artist_id = "06HL4z0CvFAxyc27GXpf02"